In [32]:
import pandas as pd
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [33]:
deaths = pd.read_csv("character-deaths.csv")
characters = pd.read_csv("character-predictions_pose.csv")
print(characters)

      S.No   plod                  name                 title  male   culture  \
0        1  0.946  Viserys II Targaryen                   NaN     1       NaN   
1        2  0.613           Walder Frey  Lord of the Crossing     1  Rivermen   
2        3  0.507          Addison Hill                   Ser     1       NaN   
3        4  0.924           Aemma Arryn                 Queen     0       NaN   
4        5  0.383        Sylva Santagar            Greenstone     0   Dornish   
...    ...    ...                   ...                   ...   ...       ...   
1941  1942  0.197                 Luwin                   NaN     1  Westeros   
1942  1943  0.013                  Reek                   NaN     1       NaN   
1943  1944  0.480      Symeon Star-Eyes                   NaN     1       NaN   
1944  1945  0.363             Coldhands                   NaN     1       NaN   
1945  1946  0.907       Tytos Lannister         Casterly Rock     1       NaN   

      dateOfBirth  DateoFde

In [34]:
characters.columns

Index(['S.No', 'plod', 'name', 'title', 'male', 'culture', 'dateOfBirth',
       'DateoFdeath', 'mother', 'father', 'heir', 'house', 'spouse', 'book1',
       'book2', 'book3', 'book4', 'book5', 'isAliveMother', 'isAliveFather',
       'isAliveHeir', 'isAliveSpouse', 'isMarried', 'isNoble', 'age',
       'numDeadRelations', 'boolDeadRelations', 'isPopular', 'popularity',
       'isAlive'],
      dtype='object')

# dataframe cleaning

In [35]:
# selected columns I think might be relevant
characters = pd.read_csv("character-predictions_pose.csv")
df = characters.loc[:,['name', 'male', 'house', 'isNoble', 'age', 'numDeadRelations', 'popularity', 'isAlive']]

# deleted all observations that have NaNs
df[~df.isnull().any(axis=1)]
df.head()

,name,male,house,isNoble,age,numDeadRelations,popularity,isAlive
0,Viserys II Targaryen,1,NaN,0,NaN,11,0.605351,0
1,Walder Frey,1,House Frey,1,97.0,1,0.896321,1
2,Addison Hill,1,House Swyft,1,NaN,0,0.267559,1
3,Aemma Arryn,0,House Arryn,1,23.0,0,0.183946,0
4,Sylva Santagar,0,House Santagar,1,29.0,0,0.043478,1


# modeling

In [36]:
model = sm.ols(formula = 'isAlive~age+male+house+isNoble+numDeadRelations+popularity', data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                isAlive   R-squared:                       0.580
Model:                            OLS   Adj. R-squared:                  0.388
Method:                 Least Squares   F-statistic:                     3.021
Date:                Sat, 18 Feb 2023   Prob (F-statistic):           2.61e-14
Time:                        16:48:45   Log-Likelihood:                -100.77
No. Observations:                 393   AIC:                             449.5
Df Residuals:                     269   BIC:                             942.3
Df Model:                         123                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            0.8148      0.048     17.001      0.000       0.720       0.909
house[T.Antler Men]                               6.992e-12   7.38e-12      0.947      0.344   -7.54e-12    2.15e-11
house[T.Band of Nine]                            -9.241e-14   1.72e-13     -0.537      0.591   -4.31e-13    2.46e-13
house[T.Black Ears]                              -1.335e-13   3.45e-13     -0.388      0.699   -8.12e-13    5.45e-13
house[T.Blacks]                                     -0.8368      0.378     -2.214      0.028      -1.581      -0.093
house[T.Brave Companions]                         3.952e-15   2.39e-13      0.017      0.987   -4.67e-13    4.75e-13
house[T.Brotherhood Without Banners]                 0.1737      0.269      0.645      0.520      -0.357       0.704
house[T.Brotherhood without Banners]              -3.04e-15   5.72e-13     -0.005      0.996   -1.13e-12    1.12e-12
house[T.Brotherhood without banners]                 0.3121      0.379      0.824      0.411      -0.433       1.058
house[T.Burned Men]                                  0.4887      0.378      1.294      0.197      -0.255       1.232
house[T.Chataya's brothel]                           0.2544      0.268      0.948      0.344      -0.274       0.783
house[T.Citadel]                                  1.007e-12   2.65e-12      0.380      0.704   -4.21e-12    6.22e-12
house[T.City Watch of King's Landing]            -1.437e-12   2.81e-12     -0.512      0.609   -6.96e-12    4.09e-12
house[T.Company of the Cat]                       2.433e-12   3.99e-12      0.610      0.542   -5.42e-12    1.03e-11
house[T.Drowned men]                             -5.106e-13      3e-12     -0.170      0.865   -6.42e-12     5.4e-12
house[T.Faceless Men]                            -2.025e-12   3.15e-12     -0.644      0.520   -8.22e-12    4.17e-12
house[T.Faith of the Seven]                          0.3851      0.220      1.750      0.081      -0.048       0.819
house[T.Golden Company]                           4.633e-12   4.06e-12      1.142      0.254   -3.35e-12    1.26e-11
house[T.Good Masters]                            -2.057e-12   5.08e-12     -0.405      0.686   -1.21e-11    7.94e-12
house[T.Graces]                                      0.3030      0.379      0.800      0.425      -0.443       1.049
house[T.Happy Port]                                  0.1760      0.378      0.466      0.642      -0.568       0.920
house[T.House Allyrion]                           3.343e-12   1.91e-12      1.750      0.081   -4.18e-13     7.1e-12
house[T.House Ambrose]                           -1.564e-12   1.84e-12     -0.850      0.396   -5.19e-12    2.06e-12
house[T.House Arryn]                     

In [37]:
model1 = sm.ols(formula = 'isAlive~age+male+house+isNoble+numDeadRelations+popularity+culture', data = characters).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                isAlive   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     2.987
Date:                Sat, 18 Feb 2023   Prob (F-statistic):           1.31e-06
Time:                        16:48:45   Log-Likelihood:                -12.057
No. Observations:                 153   AIC:                             166.1
Df Residuals:                      82   BIC:                             381.3
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                          -27.8630     14.715     -1.894      0.062     -57.136       1.410
house[T.Antler Men]                               1.676e-07   8.61e-08      1.947      0.055   -3.65e-09    3.39e-07
house[T.Band of Nine]                            -2.718e-08    1.4e-08     -1.947      0.055    -5.5e-08    5.89e-10
house[T.Black Ears]                              -3.238e-07   1.66e-07     -1.947      0.055   -6.55e-07    7.03e-09
house[T.Blacks]                                  -2.354e-07   1.21e-07     -1.947      0.055   -4.76e-07     5.1e-09
house[T.Brave Companions]                        -3.247e-08   1.67e-08     -1.947      0.055   -6.56e-08    7.03e-10
house[T.Brotherhood Without Banners]               2.21e-07   1.13e-07      1.947      0.055    -4.8e-09    4.47e-07
house[T.Brotherhood without Banners]             -6.768e-08   3.48e-08     -1.947      0.055   -1.37e-07    1.47e-09
house[T.Brotherhood without banners]              3.555e-08   1.83e-08      1.947      0.055    -7.8e-10    7.19e-08
house[T.Burned Men]                                 14.5250      7.372      1.970      0.052      -0.140      29.190
house[T.Chataya's brothel]                        1.039e-08   5.34e-09      1.946      0.055   -2.31e-10     2.1e-08
house[T.Citadel]                                 -4.213e-07   2.16e-07     -1.947      0.055   -8.52e-07    9.06e-09
house[T.City Watch of King's Landing]            -7.048e-09   3.62e-09     -1.948      0.055   -1.42e-08     1.5e-10
house[T.Company of the Cat]                        4.34e-09   2.23e-09      1.946      0.055   -9.69e-11    8.78e-09
house[T.Drowned men]                               -3.6e-07   1.85e-07     -1.947      0.055   -7.28e-07    7.81e-09
house[T.Faceless Men]                              6.32e-08   3.25e-08      1.947      0.055   -1.37e-09    1.28e-07
house[T.Faith of the Seven]                       -2.43e-10   1.25e-10     -1.947      0.055   -4.91e-10    5.27e-12
house[T.Golden Company]                           4.351e-11   2.23e-11      1.947      0.055   -9.39e-13     8.8e-11
house[T.Good Masters]                            -6.926e-11   3.56e-11     -1.947      0.055    -1.4e-10     1.5e-12
house[T.Graces]                                      9.9784      4.971      2.007      0.048       0.089      19.868
house[T.Happy Port]                                 14.4596      7.376      1.960      0.053      -0.213      29.133
house[T.House Allyrion]                           1.927e-11   9.89e-12      1.947      0.055   -4.15e-13     3.9e-11
house[T.House Ambrose]                           -2.119e-10   1.09e-10     -1.947      0.055   -4.28e-10     4.6e-12
house[T.House Arryn]                     

In [38]:
characters.culture.unique()

array([nan, 'Rivermen', 'Dornish', 'Valyrian', 'Ironborn', 'Free Folk',
       'Northmen', 'Summer Isles', 'Braavosi', 'Dothraki', 'Ghiscari',
       'Vale mountain clans', 'Reach', 'Tyroshi', 'Lhazarene',
       'Free folk', 'Ironmen', 'Qartheen', 'Lysene', 'westermen',
       'Westerman', 'Qarth', 'Lyseni', 'northmen', 'Qohor', 'Westeros',
       'Norvoshi', 'First Men', 'Meereenese', 'Andal', 'Astapori',
       'Westermen', 'ironborn', 'Ghiscaricari', 'Braavos', 'Stormlands',
       'Valemen', 'Myrish', 'Lhazareen', 'Dornishmen', 'Sistermen',
       'Northern mountain clans', 'Andals', 'Vale', 'Crannogmen',
       'Wildling', 'Dorne', 'Pentoshi', 'free folk', 'Summer Islander',
       'Westerlands', 'Summer Islands', 'Asshai', 'Riverlands', 'Naathi',
       'Rhoynar', 'Meereen', 'Norvos', 'Stormlander', 'Wildlings',
       'Astapor', 'Reachmen', "Asshai'i", 'Ibbenese', 'The Reach'],
      dtype=object)

In [39]:
culture_counts = pd.DataFrame(characters.culture.value_counts())

In [40]:
display(culture_counts)

,culture
Northmen,124
Ironborn,112
Free Folk,51
Valyrian,43
Braavosi,42
...,...
Andal,1
Norvoshi,1
Qarth,1
Lhazarene,1


In [95]:
#combining culture names that mean the same thing 
characters.culture = characters.culture.replace(to_replace = "northmen", value = "Northmen")
characters.culture = characters.culture.replace(to_replace = "ironborn", value = "Ironborn")
characters.culture = characters.culture.replace(to_replace = "Ironmen", value = "Ironborn")
characters.culture = characters.culture.replace(to_replace = "Asshai'i", value = "Asshai")
characters.culture = characters.culture.replace(to_replace = "Free folk", value = "Free Folk")
characters.culture = characters.culture.replace(to_replace = "free folk", value = "Free Folk")
characters.culture = characters.culture.replace(to_replace = "Summer Islands", value = "Summer Isles")
characters.culture = characters.culture.replace(to_replace = "Summer Islander", value = "Summer Isles")
characters.culture = characters.culture.replace(to_replace = "westermen", value = "Westermen")
characters.culture = characters.culture.replace(to_replace = "Westerman", value = "Westermen")
characters.culture = characters.culture.replace(to_replace = "Westerlands", value = "Westermen")
characters.culture = characters.culture.replace(to_replace = "Vale", value = "Valemen")
characters.culture = characters.culture.replace(to_replace = "Lhazareen", value = "Lhazarene")
characters.culture = characters.culture.replace(to_replace = "The Reach", value = "Reach")
characters.culture = characters.culture.replace(to_replace = "Reachmen", value = "Reach")
characters.culture = characters.culture.replace(to_replace = "Qarth", value = "Qartheen")
characters.culture = characters.culture.replace(to_replace = "Lyseni", value = "Lysene")
characters.culture = characters.culture.replace(to_replace = "Stormlander", value = "Stormlands")
characters.culture = characters.culture.replace(to_replace = "Meereenese", value = "Meereen")
characters.culture = characters.culture.replace(to_replace = "Astapor", value = "Astapori")
characters.culture = characters.culture.replace(to_replace = "Norvos", value = "Norvoshi")
characters.culture = characters.culture.replace(to_replace = "Wildlings", value = "Wildling")
characters.culture = characters.culture.replace(to_replace = "Andals", value = "Andal")

In [96]:
characters.culture.unique()

array([nan, 'Rivermen', 'Dornish', 'Valyrian', 'Ironborn', 'Free Folk',
       'Northmen', 'Summer Isles', 'Braavosi', 'Dothraki', 'Ghiscari',
       'Vale mountain clans', 'Reach', 'Tyroshi', 'Lhazarene', 'Qartheen',
       'Lysene', 'Westermen', 'Qohor', 'Westeros', 'Norvoshi',
       'First Men', 'Meereen', 'Andal', 'Astapori', 'Ghiscaricari',
       'Braavos', 'Stormlands', 'Valemen', 'Myrish', 'Dornishmen',
       'Sistermen', 'Northern mountain clans', 'Crannogmen', 'Wildling',
       'Dorne', 'Pentoshi', 'Asshai', 'Riverlands', 'Naathi', 'Rhoynar',
       'Ibbenese'], dtype=object)

In [97]:
culture_counts = pd.DataFrame(characters.culture.value_counts())
display(culture_counts)

,culture
Northmen,133
Ironborn,118
Free Folk,63
Valyrian,43
Braavosi,42
Ghiscari,25
Dornish,25
Dothraki,23
Valemen,20
Rivermen,19


In [98]:
model1 = sm.ols(formula = 'isAlive~age+male+house+isNoble+numDeadRelations+popularity+culture', data = characters).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                isAlive   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     3.132
Date:                Sat, 18 Feb 2023   Prob (F-statistic):           4.35e-07
Time:                        17:15:33   Log-Likelihood:                -15.282
No. Observations:                 153   AIC:                             164.6
Df Residuals:                      86   BIC:                             367.6
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                          -32.9443     16.701     -1.973      0.052     -66.144       0.256
house[T.Antler Men]                              -4.507e-07   2.23e-07     -2.018      0.047   -8.95e-07   -6.65e-09
house[T.Band of Nine]                             1.202e-08   5.96e-09      2.018      0.047    1.81e-10    2.39e-08
house[T.Black Ears]                               6.552e-07   3.25e-07      2.018      0.047    9.66e-09     1.3e-06
house[T.Blacks]                                    2.71e-07   1.34e-07      2.018      0.047       4e-09    5.38e-07
house[T.Brave Companions]                         4.625e-08   2.29e-08      2.018      0.047    6.83e-10    9.18e-08
house[T.Brotherhood Without Banners]             -4.346e-07   2.15e-07     -2.018      0.047   -8.63e-07   -6.41e-09
house[T.Brotherhood without Banners]              8.678e-08    4.3e-08      2.018      0.047    1.28e-09    1.72e-07
house[T.Brotherhood without banners]              2.882e-07   1.43e-07      2.018      0.047    4.25e-09    5.72e-07
house[T.Burned Men]                                 17.0645      8.365      2.040      0.044       0.435      33.694
house[T.Chataya's brothel]                        1.282e-06   6.35e-07      2.018      0.047    1.89e-08    2.54e-06
house[T.Citadel]                                  6.401e-07   3.17e-07      2.018      0.047    9.45e-09    1.27e-06
house[T.City Watch of King's Landing]             4.131e-07   2.05e-07      2.018      0.047     6.1e-09     8.2e-07
house[T.Company of the Cat]                      -7.943e-08   3.94e-08     -2.018      0.047   -1.58e-07   -1.18e-09
house[T.Drowned men]                              3.218e-08    1.6e-08      2.017      0.047    4.69e-10    6.39e-08
house[T.Faceless Men]                            -6.883e-08   3.41e-08     -2.018      0.047   -1.37e-07   -1.02e-09
house[T.Faith of the Seven]                       4.461e-11   2.21e-11      2.017      0.047    6.39e-13    8.86e-11
house[T.Golden Company]                            6.15e-11   3.05e-11      2.018      0.047    9.06e-13    1.22e-10
house[T.Good Masters]                            -8.982e-11   4.45e-11     -2.018      0.047   -1.78e-10   -1.35e-12
house[T.Graces]                                     11.6714      5.634      2.072      0.041       0.472      22.871
house[T.Happy Port]                                 16.9999      8.368      2.031      0.045       0.364      33.635
house[T.House Allyrion]                          -1.278e-10   6.33e-11     -2.017      0.047   -2.54e-10   -1.87e-12
house[T.House Ambrose]                            1.054e-10   5.23e-11      2.018      0.047    1.55e-12    2.09e-10
house[T.House Arryn]                     